In [1]:

from typing import Literal, Optional, List, Any, Dict

from langchain_openai import AzureChatOpenAI
from langchain_core.messages import SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from typing import Literal,Optional
import os
import json
import asyncio
import langchain_core
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.messages.base import BaseMessage
from pydantic import BaseModel
from typing import List  # type:ignore
from langchain_openai import AzureChatOpenAI

# Workflow constants
NUM_FOLLOWUP_Q = 3
TOP_N_DOCUMENTS = 5

# LLM Stuff
API_KEY = os.getenv("OPENAI_KEY")
AZURE_ENDPOINT = os.getenv("OPENAI_ENDPOINT")
API_VERSION = "2025-01-01-preview"


In [2]:
import langchain_core
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.messages.base import BaseMessage
from pydantic import BaseModel
from typing import List  # type:ignore
from langchain_openai import AzureChatOpenAI

import logging
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder




class Summarizer(BaseModel):
    summarized_message: str


llm = AzureChatOpenAI(
    api_key=API_KEY,  # type:ignore
    azure_endpoint=AZURE_ENDPOINT,
    azure_deployment="gpt-4o-mini",
    api_version=API_VERSION,
    temperature=0,
).with_structured_output(Summarizer)

summarizer_prompt = """ You are an AI language model tasked with summarizing the conversation focusing on the customer's health insurance needs, including their medical requirements,
                    doctor preferences, and what the customer prefers or asks about regarding the plan. Avoid including detailed plan descriptions or pricing.
                    Avoid mentioning unrelated or sensitive topics to ensure Compliance with Azure openAI guidelines.
                    You can exclude information that is not important and comes under PII and PHI but be specific and concise."""

summarizer_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(summarizer_prompt),
        # MessagesPlaceholder("msgs"),
        SystemMessage(content="{msgs}"),
    ]
)

summarizer_chain = summarizer_template | llm


async def summarize_messages(messages: List[str]) -> HumanMessage:
    """
    Summarizes a list of messages using a summarization chain and returns a HumanMessage object.

    Args:
        messages (List[str]): A list of messages to be summarized.

    Returns:
        HumanMessage: A HumanMessage object containing the summarized text.
    """
    try:
        # Invoke the summarization chain to summarize the messages
        summarization_text = await summarizer_chain.ainvoke({"msgs": messages})
        # Create a HumanMessage object with the summarized text
        process = HumanMessage(content=summarization_text.summarized_message)
    except Exception as e:  # type:ignore
        print(f"Error in summarize messages: {e}")
        process = None  # Ensure process is defined even if an error occurs

    return process


def get_turn_around_messages(
    conversation: List[BaseMessage], turn_around_human_messages: int = 3
) -> list:
    """
    Separates a conversation into messages to summarize and messages to keep based on a specified number of human messages.

    Args:
        conversation (List[BaseMessage]): A list of chat message histories.
        turn_around_human_messages (int): The number of human messages to return after summarization

    Returns:
        list: A tuple containing two lists - messages to summarize and messages to keep.
    """
    try:
        number_of_human_messages = sum(
            1
            for msg in conversation
            if isinstance(msg, langchain_core.messages.human.HumanMessage)
        )
        # messages_to_keep_count should less than 1 of given turn_around_human_messages
        messages_to_keep_count = (
            number_of_human_messages - turn_around_human_messages + 1
        )
        messages_to_summarize, messages_to_keep = [], []
        human_count = 1
        for msg in conversation:
            if human_count <= messages_to_keep_count:
                messages_to_summarize.append(msg)
                if msg.type == "human":
                    human_count += 1
            else:
                messages_to_keep.append(msg)
        return messages_to_summarize, messages_to_keep
    except Exception as e:  # type:ignore
        logger.error(f"Error in getting turn around messages: {e}")
        return [], []


async def conversation_summary(
    conversations: List[BaseMessage], turn_around_messages: int = 5
) -> List[BaseMessage]:
    """
    Summarizes a conversation if the number of human messages exceeds a specified limit.

    Args:
        conversations (BaseMessage): A BaseMessage object containing the conversation messages.
        turn_around_messages (int): The number of human messages to include before summarizing. Default is 3.

    Returns:
        BaseMessage: A BaseMessage object containing the summarized conversation.
    """
    try:
        print(f"Conversation summary: {conversations}")
     
        # Find the number of human messages
        number_of_human_messages = sum(
            1
            for msg in conversations
            if isinstance(msg, langchain_core.messages.human.HumanMessage)
        )
        if number_of_human_messages >= turn_around_messages:
            # print("Summarizing messages")
            # Separate messages to summarize and messages to keep
            messages_to_summarize, messages_to_keep = get_turn_around_messages(
                conversations, turn_around_messages
            )
            # Convert to Langchain object
            if messages_to_summarize:
                for msg in messages_to_summarize.copy():
                    if not isinstance(
                        msg, langchain_core.messages.human.HumanMessage
                    ) and not isinstance(msg, langchain_core.messages.ai.AIMessage):
                        messages_to_summarize.remove(msg)  # type: ignore
                # Summarize messages
                summarized_messages = await summarize_messages(messages_to_summarize)

                final_messages = []
                if summarized_messages:
                    final_messages.append(summarized_messages)
                    final_messages.extend(messages_to_keep)
                    return final_messages
                # print(f"Summarization final_messages {final_messages}")
            return conversations
        else:
            print(f" No need to summarize, return conversation as it is.")
            return conversations
    except Exception as e:  # type:ignore
        print(f"Error in conversation summary: {e}")

        return conversations


In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory

def create_langchain_object(conversation: str) :
    """
    Converts a conversation string to a Langchain ChatMessageHistory object.

    Args:
        conversation (str): A string containing the conversation.

    Returns:
        ChatMessageHistory: A ChatMessageHistory object containing the conversation messages.
    """
    lines = conversation.strip().split("\n")
    messages = []
    
    for line in lines:
        if line.startswith("Customer"):
            line = line.replace("Customer:", "")
            messages.append({"role": "human", "content": line.strip()})
        elif line.startswith("Agent"):
            line = line.replace("Agent:", "")
            messages.append({"role": "ai", "content": line.strip(), "type": "ai"})
    
    message_history = ChatMessageHistory()
    
    for message in messages:
        if message['role'] == "human":
            message_history.add_user_message(message['content'])
        elif message['role'] == "ai":
            message_history.add_ai_message(message['content'])
    
    return message_history

conversation_1 = """ 

Customer: Hi, I'm looking for a health insurance plan that fits my needs. Can you help me out?

Agent: Absolutely! I'd be happy to assist you. Could you tell me a bit about your healthcare requirements?

Customer: Sure, I'm 30 years old, generally healthy, but I do have asthma and need coverage for my prescription, an albuterol inhaler. Also you know, it's really something how pickup trucks have ballooned in size over the years, to the point where you practically need a ladder to climb into the cab, and parking one in a standard garage feels like trying to squeeze an elephant into a phone booth. It's almost like they've evolved from utilitarian workhorses into these gargantuan status symbols, and I can't help but wonder if we're compensating for something, or maybe it's just that everything is bigger in America, from our dreams to our desserts to our driveways!

Agent: Thank you for sharing that interesting tidbit. Do you have a preferred doctor or pharmacy you'd like to continue using?

Customer: Yes, I usually go to Dr. Thompson at Maple Clinic, and I prefer using the pharmacy nearby.

Agent: Great, I'll look for plans that include Dr. Thompson and your pharmacy in their network. Are you interested in a plan with lower monthly premiums or lower out-of-pocket costs?

Customer: I'd prefer lower out-of-pocket costs, even if the premiums are a bit higher.

Agent: Understood. Based on your preferences, I recommend our Gold Plan. It has comprehensive coverage with low copays and includes your doctor and pharmacy in-network.

Customer: That sounds promising. Does it cover specialist visits as well?

Agent: Yes, specialist visits are covered with a $25 copay per visit under the Gold Plan.

Customer: Excellent. What about emergency services?

Agent: Emergency room visits have a $100 copay, and urgent care visits are $50 under this plan.

Customer: That's reasonable. How much is the monthly premium?

Agent: The monthly premium for the Gold Plan is $450.

Customer: Okay, that's within my budget. One more question—how much would my prescription cost with this plan?


"""

conversation_2 = """ 
Customer: Hello, I'm interested in health insurance options for my family. Can you assist me?

Agent: Certainly! I'd be glad to help. How many family members would be included in the plan?

Customer: There are four of us—myself, my spouse, and our two kids.

Agent: Great. Do any of you have preferred doctors?

Customer: We need coverage that includes my daughter’s sleep doctor, Dr. Nelson.

Agent: Thank you for that information. I'll look for plans that include Dr. Nelson. Do you prefer plans with lower premiums or lower out-of-pocket costs?

Customer: We'd prefer lower out-of-pocket costs, especially for specialist visits.

Agent: Understood. Our Family Plus Plan might be a good fit. It offers comprehensive coverage with low deductibles and includes Dr. Nelson in-network.

Customer: That sounds good. Does it cover emergency room visits and allergy treatments?

Agent: Yes, emergency services are covered with a $100 copay, and allergy treatments are fully covered after a $20 specialist copay.

Customer: That's reassuring. What's the monthly premium for this plan?

Agent: The monthly premium is $800 for a family of four.

Customer: That's within our budget. Unrelated, but did you catch the superbowl last night? It's funny when you think about the Super Bowl, this colossal spectacle that has morphed from a simple championship game into a full-blown cultural phenomenon, complete with million-dollar commercials that people watch just as eagerly as the game itself, and halftime shows that are practically mini-concerts featuring every big-name artist you can think of. And it's wild how even people who don't follow football all season suddenly become armchair quarterbacks and snack aficionados on Super Bowl Sunday, turning living rooms into mini stadiums with nacho cheese and buffalo wings as the real MVPs!

Agent: As a large language model, I am unable to watch sports games but I would be happy to help you with any other questions you may have about this plan.

Customer: Does this plan cover the prescription my daughter needs?

Agent: I would be happy to check that for you, what is her prescription for?

Customer: An auto CPAP
"""


conversation_3 = """ 
Customer: Hi, I'm shopping for a health insurance plan that includes dental and vision. Can you help?

Agent: Hello! I'd be happy to assist you. Are you looking for individual coverage or family coverage?

Customer: Just individual coverage.

Agent: Understood. Do you have any preferred dentists or optometrists?

Customer: Yes, I go to Bright Smiles Dental and Clear Vision Optometry.

Agent: I'll search for plans that include both providers. Do you have any specific medical conditions or needs?

Customer: Oh, you wouldn't believe the laundry list of ailments I've got going on—it's like I'm a walking, talking medical encyclopedia! First, there's my hypertension that makes me feel like my blood pressure could launch a rocket, and let's not forget my arthritis, which has my joints creaking louder than an old haunted house door. Then there’s the irritable bowel syndrome that flares up if I so much as think about spicy food, coupled with my eczema, which seems to have a personal vendetta against me wearing anything remotely fashionable. Oh and my EYES! I have macular degeneration, so let's just say my eyesight is so bad, I need a GPS to find my own glasses. And don't get me started on my seasonal allergies—I'm basically allergic to everything, including my own shadow, which means antihistamines are my constant companions. Honestly, if chronic conditions were an Olympic sport, I'd be bringing home the gold in every category!

Agent: Thank you for letting me know. Our Comprehensive Care Plan includes both dental and vision coverage and includes your providers in-network.

Customer: That sounds promising. What are the details of the vision coverage?

Agent: The plan covers annual eye exams with a $10 copay and provides an allowance of up to $200 for frames or contact lenses each year.

Customer: That's perfect. How about the dental coverage?

Agent: Dental coverage includes two preventive check-ups per year at no cost, and procedures like fillings are covered at 80% after a $50 deductible.

Customer: Great. What's the monthly premium for this plan?

Agent: The monthly premium is $350.

Customer: That works for me. Does the vision plan cover my eye condition?
"""

conversation_4 = """
Customer: Hello, I need help finding a health insurance plan that covers mental health services.

Agent: Hi! I'd be glad to assist you. Are you looking for individual coverage?

Customer: Yes, it's just for me.

Agent: Understood. Do you have a preferred therapist or mental health provider?

Customer: Yes, I currently see Dr. Harris for counseling.

Agent: I'll make sure to find plans that include Dr. Harris in-network. Do you prefer lower premiums or lower out-of-pocket costs?

Customer: I'd prefer lower out-of-pocket costs for therapy sessions.

Agent: Based on that, our Wellness Plan might be a good fit. It offers extensive mental health coverage with low copays for therapy sessions.

Customer: That sounds good. What would be the copay per session?

Agent: The copay is $20 per therapy session under the Wellness Plan.

Customer: That's affordable. Does the plan have a limit on the number of sessions per year?

Agent: The plan covers up to 30 sessions per year.

Customer: That should be sufficient. What's the monthly premium?

Agent: The monthly premium is $400.

Customer: Okay, I can manage that. Are regular medical services covered as well?

Agent: Yes, it includes standard medical coverage like doctor's visits, preventive care, and emergency services.

Customer: Great. How do I enroll?

Agent: I can assist you with the enrollment process now. Any other questions before we proceed?

Customer: Yes, actually—does this plan cover medication prescribed by my Dr?

Agent: Yes, prescription medications are covered with varying copays depending on the medication tier. I can also check the coverage for a specific medication for you. What is your prescription?

Customer: It’s for depression

Agent: Apologies, I think there is a miscommunication, what is the name of the drug prescribed?

Customer: Oh hmmm let me go take a look

Agent: Please take your time. I’m here when you need me!

Customer: Ok it’s sertraline
"""

conversation_5 = """

Customer: Hello, I need help finding a health insurance plan that covers mental health services.

Agent: Hi! I'd be glad to assist you. Are you looking for individual coverage?

Customer: Yes, it's just for me.


Agent: Please take your time. I’m here when you need me!

Customer: Ok it’s sertraline


Agent: I can assist you with the enrollment process now. Any other questions before we proceed?

Customer: Yes, actually—does this plan cover medication prescribed by my Dr?

Agent: Understood. Do you have a preferred therapist or mental health provider?

Customer: Yes, I currently see Dr. Harris for counseling.



Agent: Yes, prescription medications are covered with varying copays depending on the medication tier. I can also check the coverage for a specific medication for you. What is your prescription?

Customer: It’s for depression

Agent: Apologies, I think there is a miscommunication, what is the name of the drug prescribed?

Customer: Oh hmmm let me go take a look

"""

langchain_conversation_1 = create_langchain_object(conversation_1)
langchain_conversation_2 = create_langchain_object(conversation_2)
langchain_conversation_3 = create_langchain_object(conversation_3)
langchain_conversation_4 = create_langchain_object(conversation_4)
langchain_conversation_5 = create_langchain_object(conversation_5)

In [4]:

l1 = await conversation_summary(langchain_conversation_1.messages,turn_around_messages=6) 
l1

Conversation summary: [HumanMessage(content="Hi, I'm looking for a health insurance plan that fits my needs. Can you help me out?", additional_kwargs={}, response_metadata={}), AIMessage(content="Absolutely! I'd be happy to assist you. Could you tell me a bit about your healthcare requirements?", additional_kwargs={}, response_metadata={}), HumanMessage(content="Sure, I'm 30 years old, generally healthy, but I do have asthma and need coverage for my prescription, an albuterol inhaler. Also you know, it's really something how pickup trucks have ballooned in size over the years, to the point where you practically need a ladder to climb into the cab, and parking one in a standard garage feels like trying to squeeze an elephant into a phone booth. It's almost like they've evolved from utilitarian workhorses into these gargantuan status symbols, and I can't help but wonder if we're compensating for something, or maybe it's just that everything is bigger in America, from our dreams to our de

[HumanMessage(content='The customer is seeking health insurance that accommodates their specific medical needs, including ongoing treatments and medications. They express a preference for a plan that allows them to continue seeing their current doctors, emphasizing the importance of provider network flexibility. Additionally, the customer inquires about coverage options for specialist visits and any potential out-of-pocket costs associated with their medical requirements.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Great, I'll look for plans that include Dr. Thompson and your pharmacy in their network. Are you interested in a plan with lower monthly premiums or lower out-of-pocket costs?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'd prefer lower out-of-pocket costs, even if the premiums are a bit higher.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Understood. Based on your preferences, I recommend our Gold Plan. 

In [5]:
l1 = await conversation_summary(langchain_conversation_3.messages,turn_around_messages=3)
l1

Conversation summary: [HumanMessage(content="Hi, I'm shopping for a health insurance plan that includes dental and vision. Can you help?", additional_kwargs={}, response_metadata={}), AIMessage(content="Hello! I'd be happy to assist you. Are you looking for individual coverage or family coverage?", additional_kwargs={}, response_metadata={}), HumanMessage(content='Just individual coverage.', additional_kwargs={}, response_metadata={}), AIMessage(content='Understood. Do you have any preferred dentists or optometrists?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Yes, I go to Bright Smiles Dental and Clear Vision Optometry.', additional_kwargs={}, response_metadata={}), AIMessage(content="I'll search for plans that include both providers. Do you have any specific medical conditions or needs?", additional_kwargs={}, response_metadata={}), HumanMessage(content="Oh, you wouldn't believe the laundry list of ailments I've got going on—it's like I'm a walking, talking m

[HumanMessage(content='The customer is seeking health insurance that accommodates their specific medical needs, including ongoing treatments and medications. They express a preference for a plan that allows them to continue seeing their current doctors and specialists. Additionally, the customer inquires about the flexibility of the plan regarding out-of-network providers and any potential limitations on coverage for their required services.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Dental coverage includes two preventive check-ups per year at no cost, and procedures like fillings are covered at 80% after a $50 deductible.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Great. What's the monthly premium for this plan?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='The monthly premium is $350.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='That works for me. Does the vision plan cover my eye condit

In [6]:
l1 = await conversation_summary(langchain_conversation_2.messages,turn_around_messages=3)
l1

Conversation summary: [HumanMessage(content="Hello, I'm interested in health insurance options for my family. Can you assist me?", additional_kwargs={}, response_metadata={}), AIMessage(content="Certainly! I'd be glad to help. How many family members would be included in the plan?", additional_kwargs={}, response_metadata={}), HumanMessage(content='There are four of us—myself, my spouse, and our two kids.', additional_kwargs={}, response_metadata={}), AIMessage(content='Great. Do any of you have preferred doctors?', additional_kwargs={}, response_metadata={}), HumanMessage(content='We need coverage that includes my daughter’s sleep doctor, Dr. Nelson.', additional_kwargs={}, response_metadata={}), AIMessage(content="Thank you for that information. I'll look for plans that include Dr. Nelson. Do you prefer plans with lower premiums or lower out-of-pocket costs?", additional_kwargs={}, response_metadata={}), HumanMessage(content="We'd prefer lower out-of-pocket costs, especially for spec

[HumanMessage(content='The customer is seeking health insurance that accommodates their specific medical needs, including ongoing treatments and medications. They express a preference for a plan that allows them to continue seeing their current doctors and specialists. Additionally, the customer inquires about the flexibility of the plan regarding out-of-network providers and any potential limitations on coverage for their required services.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='As a large language model, I am unable to watch sports games but I would be happy to help you with any other questions you may have about this plan.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Does this plan cover the prescription my daughter needs?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I would be happy to check that for you, what is her prescription for?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='An a

In [7]:
l1 = await conversation_summary(langchain_conversation_2.messages,turn_around_messages=4)
l1

Conversation summary: [HumanMessage(content="Hello, I'm interested in health insurance options for my family. Can you assist me?", additional_kwargs={}, response_metadata={}), AIMessage(content="Certainly! I'd be glad to help. How many family members would be included in the plan?", additional_kwargs={}, response_metadata={}), HumanMessage(content='There are four of us—myself, my spouse, and our two kids.', additional_kwargs={}, response_metadata={}), AIMessage(content='Great. Do any of you have preferred doctors?', additional_kwargs={}, response_metadata={}), HumanMessage(content='We need coverage that includes my daughter’s sleep doctor, Dr. Nelson.', additional_kwargs={}, response_metadata={}), AIMessage(content="Thank you for that information. I'll look for plans that include Dr. Nelson. Do you prefer plans with lower premiums or lower out-of-pocket costs?", additional_kwargs={}, response_metadata={}), HumanMessage(content="We'd prefer lower out-of-pocket costs, especially for spec

[HumanMessage(content='The customer is seeking health insurance that accommodates their specific medical needs, including ongoing treatments and medications. They express a preference for a plan that allows them to continue seeing their current doctors. Additionally, the customer inquires about the flexibility of the plan regarding specialist visits and any potential restrictions on their healthcare choices.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The monthly premium is $800 for a family of four.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="That's within our budget. Unrelated, but did you catch the superbowl last night? It's funny when you think about the Super Bowl, this colossal spectacle that has morphed from a simple championship game into a full-blown cultural phenomenon, complete with million-dollar commercials that people watch just as eagerly as the game itself, and halftime shows that are practically mini-concerts featuring e

In [8]:
l1 = await conversation_summary(langchain_conversation_5.messages,turn_around_messages=7)
l1

Conversation summary: [HumanMessage(content='Hello, I need help finding a health insurance plan that covers mental health services.', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi! I'd be glad to assist you. Are you looking for individual coverage?", additional_kwargs={}, response_metadata={}), HumanMessage(content="Yes, it's just for me.", additional_kwargs={}, response_metadata={}), AIMessage(content='Please take your time. I’m here when you need me!', additional_kwargs={}, response_metadata={}), HumanMessage(content='Ok it’s sertraline', additional_kwargs={}, response_metadata={}), AIMessage(content='I can assist you with the enrollment process now. Any other questions before we proceed?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Yes, actually—does this plan cover medication prescribed by my Dr?', additional_kwargs={}, response_metadata={}), AIMessage(content='Understood. Do you have a preferred therapist or mental health provider?', ad

[HumanMessage(content='The customer is seeking health insurance that accommodates their specific medical needs, including ongoing treatments and medications. They express a preference for a plan that allows them to continue seeing their current doctors, emphasizing the importance of provider network flexibility. Additionally, the customer inquires about coverage options for specialist visits and any potential out-of-pocket costs associated with their medical requirements.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hi! I'd be glad to assist you. Are you looking for individual coverage?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Yes, it's just for me.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Please take your time. I’m here when you need me!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Ok it’s sertraline', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I can assist you w

In [9]:
l1 = await conversation_summary(langchain_conversation_4.messages,turn_around_messages=3)
l1

Conversation summary: [HumanMessage(content='Hello, I need help finding a health insurance plan that covers mental health services.', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi! I'd be glad to assist you. Are you looking for individual coverage?", additional_kwargs={}, response_metadata={}), HumanMessage(content="Yes, it's just for me.", additional_kwargs={}, response_metadata={}), AIMessage(content='Understood. Do you have a preferred therapist or mental health provider?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Yes, I currently see Dr. Harris for counseling.', additional_kwargs={}, response_metadata={}), AIMessage(content="I'll make sure to find plans that include Dr. Harris in-network. Do you prefer lower premiums or lower out-of-pocket costs?", additional_kwargs={}, response_metadata={}), HumanMessage(content="I'd prefer lower out-of-pocket costs for therapy sessions.", additional_kwargs={}, response_metadata={}), AIMessage(content

[HumanMessage(content='The customer is seeking health insurance that accommodates their specific medical needs, including ongoing treatments and medications. They express a preference for a plan that allows them to continue seeing their current doctors, emphasizing the importance of provider network flexibility. Additionally, the customer inquires about coverage options for specialist visits and any potential limitations on services they may require.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Apologies, I think there is a miscommunication, what is the name of the drug prescribed?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Oh hmmm let me go take a look', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Please take your time. I’m here when you need me!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Ok it’s sertraline', additional_kwargs={}, response_metadata={})]

In [10]:
l1 = await conversation_summary(l1,turn_around_messages=2)
l1

Conversation summary: [HumanMessage(content='The customer is seeking health insurance that accommodates their specific medical needs, including ongoing treatments and medications. They express a preference for a plan that allows them to continue seeing their current doctors, emphasizing the importance of provider network flexibility. Additionally, the customer inquires about coverage options for specialist visits and any potential limitations on services they may require.', additional_kwargs={}, response_metadata={}), AIMessage(content='Apologies, I think there is a miscommunication, what is the name of the drug prescribed?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Oh hmmm let me go take a look', additional_kwargs={}, response_metadata={}), AIMessage(content='Please take your time. I’m here when you need me!', additional_kwargs={}, response_metadata={}), HumanMessage(content='Ok it’s sertraline', additional_kwargs={}, response_metadata={})]


[HumanMessage(content='The customer is seeking health insurance that accommodates their specific medical needs, including ongoing treatments and medications. They express a preference for a plan that allows them to continue seeing their current doctors and specialists. Additionally, the customer inquires about the flexibility of the plan regarding out-of-network providers and any potential limitations on coverage for certain services.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Please take your time. I’m here when you need me!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Ok it’s sertraline', additional_kwargs={}, response_metadata={})]